In [27]:
# CELL 0 — Detect Colab + Config

import os, sys

def is_colab():
    return "google.colab" in sys.modules

IN_COLAB = is_colab()
USE_NGROK = True  # bật/tắt tunnel tùy nhu cầu

print("🧪 IN_COLAB:", IN_COLAB)
print("🚇 USE_NGROK:", USE_NGROK)


🧪 IN_COLAB: True
🚇 USE_NGROK: True


In [3]:
# CELL 1: Install dependencies
!pip install fastapi uvicorn[standard] transformers accelerate huggingface_hub safetensors
!pip install nest_asyncio psutil

import nest_asyncio
nest_asyncio.apply()


In [13]:
!pip install -q pyngrok

In [18]:
os.environ["NGROK_AUTH_TOKEN"] = "369ihQkE1qSAcBF8zOwQltYwTMm_5L3jbYXmDkdS9mefmLHt6"


In [19]:
# CELL 2 — ENV CONFIG + AUTO PORT + NGROK_AUTH_TOKEN

import socket, os
from pyngrok import ngrok
from google.colab import drive

def find_free_port():
    s = socket.socket()
    s.bind(('', 0))
    p = s.getsockname()[1]
    s.close()
    return p

# Port ngẫu nhiên để tránh conflict
PORT = find_free_port()
print(f"🌐 Free port được chọn: {PORT}")

# HuggingFace token (nếu bạn dùng)
HF_TOKEN = os.environ.get("HF_HUB_TOKEN", "").strip()

# Token Ngrok (v2 compatible)
NGROK_AUTH_TOKEN = os.environ.get("NGROK_AUTH_TOKEN", "").strip()

if not NGROK_AUTH_TOKEN:
    raise ValueError(
        "❌ Bạn chưa đặt biến môi trường NGROK_AUTH_TOKEN.\n"
        "Hãy chạy: os.environ['NGROK_AUTH_TOKEN'] = 'xxxx'"
    )

print("🔑 Token đã lấy từ môi trường:", NGROK_AUTH_TOKEN[:6] + "******")

# Gán token vào pyngrok
ngrok.set_auth_token(NGROK_AUTH_TOKEN)

print("✔️ Cell 2 hoàn tất.")


🌐 Free port được chọn: 48757
🔑 Token đã lấy từ môi trường: 369ihQ******
✔️ Cell 2 hoàn tất.


In [21]:
# CELL 3: Load HuggingFace model usho/Llama-3.2-3B

from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from huggingface_hub import login
import torch

HF_MODEL = "unsloth/Llama-3.2-3B"

if HF_TOKEN:
    login(HF_TOKEN)

print("🚀 Đang tải mô hình:", HF_MODEL)

tokenizer = AutoTokenizer.from_pretrained(HF_MODEL, use_fast=True)

model = AutoModelForCausalLM.from_pretrained(
    HF_MODEL,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32,
    device_map="auto",
    trust_remote_code=True
)

text_gen = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    return_full_text=False
)

print("✔️ usho/Llama-3.2-3B đã load xong")


🚀 Đang tải mô hình: unsloth/Llama-3.2-3B


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/459 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/890 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/230 [00:00<?, ?B/s]

Device set to use cuda:0


✔️ usho/Llama-3.2-3B đã load xong


In [25]:
# CELL: Mount Google Drive + set HF cache

from google.colab import drive
drive.mount('/content/drive')

import os

# Thư mục chứa model cache
HF_CACHE_DIR = "/content/drive/MyDrive/hf_models"

# Tạo thư mục nếu chưa có
os.makedirs(HF_CACHE_DIR, exist_ok=True)

# Cấu hình HuggingFace để cache vào Drive
os.environ["HF_HOME"] = HF_CACHE_DIR
os.environ["TRANSFORMERS_CACHE"] = HF_CACHE_DIR
os.environ["HF_DATASETS_CACHE"] = HF_CACHE_DIR

print("📁 HuggingFace cache folder:", HF_CACHE_DIR)
print("✔️ Tất cả mô hình sẽ được tải vào Drive để sử dụng lại sau.")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
📁 HuggingFace cache folder: /content/drive/MyDrive/hf_models
✔️ Tất cả mô hình sẽ được tải vào Drive để sử dụng lại sau.


In [22]:
# CELL 4: FastAPI server

from fastapi import FastAPI, Request
import uvicorn

app = FastAPI()

def build_prompt(messages):
    prompt = ""
    for m in messages:
        role = m.get("role")
        content = m.get("content", "")
        if role == "system":
            prompt += f"[SYSTEM] {content}\n"
        elif role == "user":
            prompt += f"User: {content}\n"
        else:
            prompt += f"Assistant: {content}\n"
    return prompt + "Assistant:"

@app.post("/v1/chat/completions")
async def chat_completions(request: Request):
    data = await request.json()
    messages = data.get("messages", [])
    temperature = data.get("temperature", 0.2)
    max_tokens = data.get("max_tokens", 256)

    prompt = build_prompt(messages)

    generated = text_gen(
        prompt,
        max_new_tokens=max_tokens,
        temperature=temperature
    )

    reply = generated[0]["generated_text"]

    if reply.startswith(prompt):
        reply = reply[len(prompt):].strip()

    return {
        "choices": [
            {
                "message": {
                    "role": "assistant",
                    "content": reply
                }
            }
        ]
    }

print(f"🚀 FastAPI ready on port {PORT}")


🚀 FastAPI ready on port 48757


In [28]:
# CELL 5 — START SERVER with PYNGROK (v2)

print("--- Khởi Chạy Server ---")

if IN_COLAB and USE_NGROK:
    try:
        token = NGROK_AUTH_TOKEN  # lấy từ cell 2
        if token:
            print("🔑 NGROK_AUTH_TOKEN đã sẵn sàng.")

            # Kill process cũ để tránh lỗi ERR_NGROK_105
            ngrok.kill()

            # Mở tunnel cho PORT đã chọn
            public_url = ngrok.connect(PORT)
            print(f"✅ Server URL (Ngrok): {public_url.public_url}")
            print(f"👉 Copy URL này dán vào Agent.")
        else:
            print("❌ Không tìm thấy token")

    except Exception as e:
        print(f"⚠️ Lỗi Ngrok: {e}")
        print("⚠️ Server sẽ chạy Localhost")

else:
    print("🌐 Không chạy Colab hoặc USE_NGROK = False → Server chạy Localhost")


--- Khởi Chạy Server ---
🔑 NGROK_AUTH_TOKEN đã sẵn sàng.
✅ Server URL (Ngrok): https://elissa-villous-scourgingly.ngrok-free.dev
👉 Copy URL này dán vào Agent.
